# Example usage

To use `dbpd` in a project:

In [1]:
# Automatically reload modules
%load_ext autoreload
%autoreload 2

In [2]:
import dbpd
print(dbpd.__version__)

0.1.0


### Load the metadata and binaries

In [3]:
import tsdf
import numpy as np
import pandas as pd
from dbpd.constants import DataColumns

metadata_dict = tsdf.load_metadata_from_path('../data/WatchData.IMU.Week0.raw_segment0161_meta.json')

# Retrieve the metadata object we want, using the name of the binary as key
metadata_samples = metadata_dict["WatchData.IMU.Week0.raw_segment0161_samples.bin"]
data = tsdf.load_binary_from_metadata(metadata_samples)
metadata_time = metadata_dict["WatchData.IMU.Week0.raw_segment0161_time.bin"]
time = tsdf.load_binary_from_metadata(metadata_time)

# Print some info
print(f"Data shape:\t {data.shape}")
print(time[0:10])

Data shape:	 (6757787, 6)
[0.        9.8532715 9.853516  9.8532715 9.8532715 9.8532715 9.853516
 9.8532715 9.8532715 9.8532715]


#### Setup a preprocessing configuration

In [4]:
processingConfig = dbpd.PreprocessingPipelineConfig(DataColumns.TIME, 100, 100, True)

### Convert time from delta time to relative time

In [5]:
time = dbpd.transform_time_array(time, 1000, True)
print(time[0:10])

[0.         0.00985327 0.01970679 0.02956006 0.03941333 0.0492666
 0.05912012 0.06897339 0.07882666 0.08867993]


#### Resample the data


In [6]:
df = dbpd.resample_data(processingConfig, time, data, metadata_samples.scale_factors)
print(df.shape)
df.head()

(6662096, 7)


,time,acceleration_x,acceleration_y,acceleration_z,rotation_x,rotation_y,rotation_z
0,0.00,-2.689536,-10.931814,1.042019,247.987806,-270.914635,196.341464
1,0.01,-1.588193,-10.467764,-4.220596,242.813985,-272.776486,196.123144
2,0.02,-1.308063,-11.668771,-4.109930,207.835635,-260.688301,187.506886
3,0.03,-1.311445,-10.683093,-2.101027,156.162746,-219.600694,170.990978
4,0.04,-2.464753,-10.089515,0.602694,127.339890,-220.812595,178.438022
